In [4]:
# Importing the necessary libraries for the data analysis and transformations
import pandas as pd
import numpy as np
from dask import dataframe as dd
import matplotlib.pyplot as plt
from sklearn import preprocessing, decomposition, impute
from sklearn.experimental import enable_iterative_imputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import confusion_matrix
!pip install patool
from pyunpack import Archive
import shutil

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install py7zr

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
#!pip install patool
# extract the folds into folds folder
import py7zr
with py7zr.SevenZipFile('../../../data/folds.7z', mode='r') as z:
    z.extractall(path='folds/')

In [7]:
#Obtaining the collapsed dataset
fold_1 = pd.read_csv('./folds/fold_1.csv')
fold_2 = pd.read_csv('./folds/fold_2.csv')
fold_3 = pd.read_csv('./folds/fold_3.csv')

In [8]:
fold_1.append(fold_2)
fold_1.append(fold_3)
fold_1
all_folds = fold_1
all_folds

C:\Users\bobad\AppData\Local\Temp\ipykernel_14700\480925214.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_1.append(fold_2)
C:\Users\bobad\AppData\Local\Temp\ipykernel_14700\480925214.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_1.append(fold_3)


,Unnamed: 0,SOCIO_ECONOMIC_SEGMENT,PARTY_GENDER_CD,BILL_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,POSTPAID_LINES,OTHER_LINES,STATUS,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,Years_stayed,TARGET
0,0,1.272502,0.490663,0.878091,0.474766,0.054791,0.247272,0.206937,0.344986,0.272202,0.371069,0.432914,0.445148,0.301507,0.446350,0.192129,0.312506,2.855943,0.0
1,1,1.855048,1.969370,0.269332,0.474766,0.046936,0.288660,0.530613,0.344986,0.272202,0.106462,0.432914,0.089731,0.301507,0.446350,0.192129,0.394721,0.584268,0.0
2,2,1.272502,1.969370,0.087057,0.474766,0.044215,0.288660,0.116740,0.344986,0.272202,0.251207,1.309613,0.178970,0.036861,0.446350,0.186231,0.123189,0.256629,0.0
3,3,1.855048,0.490663,0.310871,0.474766,0.077323,0.556627,0.440417,0.344986,0.272202,0.128043,0.275344,1.324676,0.020072,0.446350,0.192129,0.256665,0.256629,0.0
4,4,1.272502,0.490663,0.135277,0.474766,0.061506,0.020694,0.206937,0.344986,0.272202,0.226235,1.051840,0.043717,0.168134,0.446350,0.192129,0.244986,2.855943,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266136,266136,0.291273,0.490663,0.429144,0.474766,0.016058,0.288660,0.116740,0.135102,0.272202,0.311009,0.251142,0.225443,0.270262,0.436389,0.192129,0.284774,0.748087,0.0
266137,266137,0.291273,0.490663,0.357121,0.474766,0.087923,0.515239,0.440417,0.344986,0.272202,0.050930,0.432914,0.361544,0.301507,0.197516,0.192129,0.225262,0.748087,0.0
266138,266138,1.272502,0.490663,0.854905,0.474766,0.005053,0.020694,1.501643,0.344986,0.272202,0.430497,0.395466,0.334972,0.292342,0.446350,0.192129,0.312503,0.748087,1.0
266139,266139,0.291273,0.490663,0.124640,0.893708,0.070680,0.783205,0.440417,0.344986,0.272202,0.170523,0.041721,0.110920,0.098744,0.411182,0.181878,0.036555,0.092809,0.0


In [9]:
#Defining dependant and independant variables
Y = all_folds['TARGET'] #Dependant
X = all_folds.drop(columns=['TARGET', 'Unnamed: 0']) #Independant

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [11]:
model = Sequential()
model.add(Dense(50, input_shape=(17,), activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, Y, epochs=10, batch_size=64, workers=12)
# evaluate the keras model
_, accuracy = model.evaluate(X, Y)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/10
 945/4159 [=====>........................] - ETA: 9s - loss: 0.2088 - accuracy: 0.9459